# CHRONOS - Toy
> Versión 8 octubre

In [ ]:
#! pip install git+https://github.com/amazon-science/chronos-forecasting.git

In [1]:
import pandas as pd  # requires: pip install pandas
import torch
from chronos import ChronosPipeline
from dvats.all import *

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib.


Octave is ready <oct2py.core.Oct2Py object at 0x7fa946fe6d70>

















## Load model

In [4]:
?? ChronosPipeline.from_pretrained

Signature:  ChronosPipeline.from_pretrained(*args, **kwargs)
Source:   
    @classmethod
    def from_pretrained(cls, *args, **kwargs):
        """
        Load the model, either from a local path or from the HuggingFace Hub.
        Supports the same arguments as ``AutoConfig`` and ``AutoModel``
        from ``transformers``.
        """

        config = AutoConfig.from_pretrained(*args, **kwargs)

        assert hasattr(config, "chronos_config"), "Not a Chronos config file"

        chronos_config = ChronosConfig(**config.chronos_config)

        if chronos_config.model_type == "seq2seq":
            inner_model = AutoModelForSeq2SeqLM.from_pretrained(*args, **kwargs)
        else:
            assert chronos_config.model_type == "causal"
            inner_model = AutoModelForCausalLM.from_pretrained(*args, **kwargs)

        return cls(
            tokenizer=chronos_config.create_tokenizer(),
            model=ChronosModel(config=chronos_config, model=inner_model),
        )
File:  

In [5]:
model_name                    = "chronos"
model_size                    = "small"
model_url                     = f"amazon/chronos-1-{model_size}"
model_task                    = ""

In [7]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16,
)

In [12]:
#? ChronosPipeline

In [11]:
model = pipeline.model

## Get the dataset

In [13]:
#| export
verbose                       = 1  
show_plots                    = False
reset_kernel                  = False
pre_configured_case           = True 
case_id                       = 7 
frequency_factor              = 1
frequency_factor_change_alias = True 
cuda_device                   = 0

In [21]:
#| export
import pandas as pd
import numpy as np
from fastcore.all import *
import wandb
from dvats.load import TSArtifact, infer_or_inject_freq
import pickle
import matplotlib
import matplotlib.pyplot as plt
from tsai.data.external import convert_tsf_to_dataframe
from tsai.utils import stack_pad
#| export
import warnings
warnings.filterwarnings("ignore", module="umap")
import os
import sys
sys.path.append(os.path.abspath('..'))
from dvats.all import *
from fastcore.all import *
from tsai.basics import *
from tsai.models.InceptionTimePlus import *
from tsai.callback.MVP import *
import matplotlib.colors as colors
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from fastai.callback.schedule import *
from fastai.callback.tracker import EarlyStoppingCallback
import wandb

import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import sys
import dvats.utils as ut
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

import uni2ts
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_next_multi
import pyarrow.feather as ft
from gluonts.transform.split import TFTInstanceSplitter
from gluonts.transform.sampler import TestSplitSampler
import numpy as np
import einops
import torch.nn.functional as F
from dvats.memory import gpu_memory_status

## Load dataset & apply sliding window

In [33]:
#| export
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)
if pre_configured_case: 
    cfg_.force_artifact_config_mvp(
        config = config,
        id = case_id,
        verbose = verbose, 
        both = verbose > 0,
        frequency_factor = frequency_factor,
        frequency_factor_change_alias = frequency_factor_change_alias
    )
#| export
path = os.path.expanduser("~/work/nbs_pipeline/")
name="02c_encoder_moment-embedding"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
if verbose > 0: print("runname: "+runname)
if verbose > 0: cfg_.show_attrdict(config)

#| export
if verbose > 0: print("--> Wandb init")
run = wandb.init(
    entity = user,
    # work-nbs is a place to log draft runs
    project=project,
    group=config.wandb_group,
    job_type=job_type,
    allow_val_change=True,
    mode=config.analysis_mode,
    config=config,
    # When use_wandb is false the run is not linked to a personal account
    #NOTE: This is not working right now
    anonymous = 'never' if config.use_wandb else 'must', 
    resume=False,
    name = runname
)
if verbose > 0: print("Wandb init -->")
config = run.config  # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
#| export
config = run.config  # Object for storing hyperparameters
if verbose > 0: cfg_.show_attrdict(config)
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
train_artifact = artifacts_gettr(config.train_artifact)
if verbose > 0: print("---> W&B Train Artifact")

#| export
import pyarrow.feather as ft
df_train = train_artifact.to_df()
print("df_train ~", df_train.shape)
print(df_train.shape)
display(df_train.head())
#| export
if verbose > 0: print("---> Sliding window | ", config.w,  " | ", config.stride )
sw = SlidingWindow(window_len=config.w, stride=config.stride, get_y=[])
if verbose > 0: print(" Sliding window | ", config.w,  " | ", config.stride, "---> | df_train ~ ", df_train.shape )
X_train, _ = sw(df_train)
if verbose > 0: print(" sw_df_train | ", config.w,  " | ", config.stride, "--->" )
print("X_train ~", X_train.shape)

valid_artifact: None
stride: 900 -> 1
mvp_ws: (15, 100) -> [10, 30]
train_artifact: mi-santamaria/deepvats/PulsusParadoxus-SP02:latest -> mi-santamaria/deepvats/toy:latest
alias: PulsusParadoxus-SP02 -> toy
norm_by_sample: False
norm_use_single_batch: False
epochs: 100
mask_sync: False
time_col is missing in original dict | None 
mask_future: False
artifact_name is missing in original dict | toy 
wandb_group: None
w: 100 -> 30
csv_config is missing in original dict | {} 
freq is missing in original dict | 1s 
norm_use_by_single_batch is missing in original dict | (False,) 
r: 0.71
mask_stateful: True
data_cols is missing in original dict | [] 
analysis_mode: online
data_fpath is missing in original dict | ~/data/toy.csv 
batch_size: 512 -> 32
valid_size: 0.2
use_wandb: True
runname: 02c_encoder_moment-embedding
alias: toy
analysis_mode: online
batch_size: 32
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [10, 30]
norm_by_sample: False
norm_use_single_batch:

Wandb init -->
alias: toy
analysis_mode: online
batch_size: 32
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [10, 30]
norm_by_sample: False
norm_use_single_batch: False
r: 0.71
stride: 1
train_artifact: mi-santamaria/deepvats/toy:latest
valid_artifact: None
use_wandb: True
valid_size: 0.2
w: 30
wandb_group: None
artifact_name: toy
data_cols: []
data_fpath: ~/data/toy.csv
freq: 1s
time_col: None
csv_config: {}
norm_use_by_single_batch: [False]
---> W&B Train Artifact


wandb:   1 of 1 files downloaded.  


df_train ~ (550, 3)
(550, 3)


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


---> Sliding window |  30  |  1
 Sliding window |  30  |  1 ---> | df_train ~  (550, 3)
 sw_df_train |  30  |  1 --->
X_train ~ (521, 3, 30)


In [22]:
print(f"X_train ~ {X_train.shape}")

X_train ~ (521, 3, 30)


In [40]:
X_train_splitted = einops.rearrange(X_train, 'b c d -> c b d')
context = [torch.from_numpy(X_train_splitted[i]) for i in range(3)]

In [42]:
for i in range (len(context)):
    print(f"Context tensor {i} ~ {context[i].shape}")

Context tensor 0 ~ torch.Size([521, 30])
Context tensor 1 ~ torch.Size([521, 30])
Context tensor 2 ~ torch.Size([521, 30])


In [26]:
#? model

In [43]:
( 
    input_ids, 
    attention_mask, 
    _
) = pipeline.tokenizer.context_input_transform(
    context
)

AttributeError: 'list' object has no attribute 'shape'

In [45]:
context2 = [df["T1"], df["T2"], df["T3"]]
for i in range (len(context)):
    print(f"Context tensor {i} ~ {context2[i].shape}")

Context tensor 0 ~ (550,)
Context tensor 1 ~ (550,)
Context tensor 2 ~ (550,)


In [46]:
pipeline.predict(
    context = context2
)

AssertionError: 

In [24]:
? model.forward

Signature:
 model.forward(
    input_ids: torch.Tensor,
    attention_mask: torch.Tensor,
    prediction_length: Optional[int] = None,
    num_samples: Optional[int] = None,
    temperature: Optional[float] = None,
    top_k: Optional[int] = None,
    top_p: Optional[float] = None,
) -> torch.Tensor
Docstring:
Predict future sample tokens for the given token sequences.

Arguments ``prediction_length``, ``num_samples``, ``temperature``,
``top_k``, ``top_p`` can be used to customize the model inference,
and default to the corresponding attributes in ``self.config`` if
not provided.

Returns
-------
samples
    A tensor of integers, shaped (batch_size, num_samples, time_length),
    containing forecasted sample paths.
File:      ~/.local/lib/python3.10/site-packages/chronos/chronos.py
Type:      method